In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/world-happiness/2019.csv")

In [ ]:
df

In [ ]:
# Mutluluk oranı 6.0 ve üzeri olan ülkeri mutlu olarak saydık
# Diğerlerini mutsuz olarak saydık
y = df["Score"].apply(lambda x : int(x / 6))

In [ ]:
df.columns

In [ ]:
x = df.drop(columns = ['Overall rank', 'Score', 'Country or region'])
# normalleştirme işlemi
x = (x - np.min(x)) / (np.max(x) - np.min(x))
x

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(x_train, y_train)
logistic.score(x_test, y_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, logistic.predict(x_test))

def oran_dondur(cm):
    mutsuz_oran = cm[0][0] / (cm[0][0] + cm[0][1])
    mutlu_oran = cm[1][1] / (cm[1][0] + cm[1][1])
    return [mutlu_oran, mutsuz_oran]

oranlar = {}
oranlar["logistic"] = oran_dondur(cm)
oranlar

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
best_n = 0
score = 0
# En iyi sonucu almak için en ideal komşu sayısını seçtik
for i in range(1, 100):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    new_score = knn.score(x_test, y_test)
    if new_score > score:
        score = new_score
        best_n = i
        cm = confusion_matrix(y_test, knn.predict(x_test))
oranlar["KNN"] = oran_dondur(cm)
oranlar

In [ ]:
# Support Vector Machine
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train, y_train)
cm = confusion_matrix(y_test, svc.predict(x_test))
oranlar["SVC"] = oran_dondur(cm)
oranlar

In [ ]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(x_train, y_train)
cm = confusion_matrix(y_test, naive.predict(x_test))
oranlar["Naive"] = oran_dondur(cm)
oranlar

In [ ]:
# Random Forest 
from sklearn.ensemble import RandomForestClassifier
best_n = 1
score = 0
for i in range(1,50):
    rforest = RandomForestClassifier(n_estimators=i)
    rforest.fit(x_train, y_train)
    new_score = rforest.score(x_test, y_test)
    if new_score > score:
        score = new_score
        best_n = i
        cm = confusion_matrix(y_test, rforest.predict(x_test))
oranlar["R-Forest"] = oran_dondur(cm)
oranlar

In [ ]:
names = []
mutlu_doğru = [] # Mutlu olan ülkeler arasında doğru tahmin oranı
mutsuz_doğru = [] # Mutsuz olan ülkeler arasında doğru tahim oranı
for i in oranlar.items():
    names.append(i[0])
    mutlu_doğru.append(i[1][0]*100)
    mutsuz_doğru.append(i[1][1]*100)
mutlu_doğru

In [ ]:
from matplotlib import pyplot as plt
plt.bar(np.arange(5) - 0.3, mutlu_doğru, 0.3, label = "Mutlu")
plt.bar(np.arange(5), mutsuz_doğru, 0.3, label = "Mutsuz")
plt.legend()
plt.xlabel("Öğrenme Tipi")
plt.ylabel("Doğruluk Oranı")
plt.xticks(np.arange(5), labels=names)
plt.show()